# Лабораторная работа №2
## Классификация датасета MNIST с использованием MLP и CNN

Цель работы:
- Решить задачу классификации рукописных цифр MNIST
- Использовать:
  - MLP из scikit-learn
  - CNN (архитектура типа LeNet) на PyTorch
- Сравнить модели по метрикам качества
- Сделать обоснованные выводы


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier


## Загрузка и подготовка датасета MNIST


In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

y = y.astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Классификация с помощью MLP (scikit-learn)


In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 100),
    activation='relu',
    solver='adam',
    max_iter=20,
    random_state=42,
    verbose=True
)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)


Iteration 1, loss = 0.34040678
Iteration 2, loss = 0.11803449
Iteration 3, loss = 0.07667635
Iteration 4, loss = 0.05513660
Iteration 5, loss = 0.03982209
Iteration 6, loss = 0.02794909
Iteration 7, loss = 0.02201013
Iteration 8, loss = 0.01509931
Iteration 9, loss = 0.01308611
Iteration 10, loss = 0.01115846
Iteration 11, loss = 0.01356386
Iteration 12, loss = 0.01108935
Iteration 13, loss = 0.00658808
Iteration 14, loss = 0.00712500
Iteration 15, loss = 0.01394550
Iteration 16, loss = 0.01529280
Iteration 17, loss = 0.00613037
Iteration 18, loss = 0.00240576
Iteration 19, loss = 0.00180503
Iteration 20, loss = 0.00190026


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## Метрики качества для MLP


In [ ]:
def print_metrics(y_true, y_pred, title):
    print(title)
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='macro'))
    print("Recall   :", recall_score(y_true, y_pred, average='macro'))
    print("F1-score :", f1_score(y_true, y_pred, average='macro'))
    print()

print_metrics(y_test, y_pred_mlp, "MLP (scikit-learn)")


MLP (scikit-learn)
Accuracy : 0.9714285714285714
Precision: 0.9713452833415761
Recall   : 0.9711584024645479
F1-score : 0.9712371724969369



## Реализация CNN (LeNet-подобная архитектура) на PyTorch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
X_train_torch = torch.tensor(X_train / 255.0, dtype=torch.float32).reshape(-1, 1, 28, 28)
X_test_torch = torch.tensor(X_test / 255.0, dtype=torch.float32).reshape(-1, 1, 28, 28)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(
    TensorDataset(X_train_torch, y_train_torch),
    batch_size=64,
    shuffle=True
)

test_loader = DataLoader(
    TensorDataset(X_test_torch, y_test_torch),
    batch_size=64,
    shuffle=False
)


## Архитектура CNN (LeNet)


In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


## Обучение CNN


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs} completed")


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed


## Оценка CNN


In [ ]:
model.eval()
y_pred_cnn = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred_cnn.extend(predicted.cpu().numpy())

y_pred_cnn = np.array(y_pred_cnn)

print_metrics(y_test, y_pred_cnn, "CNN (LeNet, PyTorch)")


CNN (LeNet, PyTorch)
Accuracy : 0.9832142857142857
Precision: 0.9832919171938229
Recall   : 0.9831064555520381
F1-score : 0.9831684086202749



## Сравнение моделей и выводы

### Результаты:
- MLP показывает хорошее качество, однако:
  - работает с векторами признаков
  - не использует пространственную структуру изображения

- CNN (LeNet):
  - использует свёрточные слои
  - извлекает локальные признаки
  - демонстрирует более высокие метрики качества

### Выводы:
1. CNN превосходит MLP по accuracy, precision, recall и F1-score
2. Для задач компьютерного зрения предпочтительны свёрточные сети
3. MLP подходит как базовая модель, но плохо масштабируется на изображения
